In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import umap
import matplotlib.pyplot as plt

from tqdm import tqdm

import torch
from torch.utils.data import DataLoader

import pytorch_lightning as pl

from deeptime.models.oneclass.conv import ConvOCC
from deeptime.models.representation import ConvAutoEncoder
from deeptime.data import BaseDataset

from sktime.datasets import load_UCR_UEA_dataset

from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
DATASETS = [
    'Yoga',
    # 'WormsTwoClass',
    # 'Wine',
    # 'Wafer',
    # 'TwoLeadECG',
    # 'Strawberry',
    # 'SemgHandGenderCh2', 
    # 'BeetleFly',
    # 'BirdChicken',
    # 'Computers',
    # 'DistalPhalanxOutlineCorrect',
    # 'Earthquakes',
    # 'ECG200',
    # 'ECGFiveDays',
    # 'FordA',
    # 'FordB',
    # 'HandOutlines',
    # 'ItalyPowerDemand', 
    # 'MiddlePhalanxOutlineCorrect',
    # 'Chinatown',
    # 'FreezerRegularTrain',
    # 'FreezerSmallTrain',
    # 'GunPointAgeSpan',
    # 'GunPointMaleVersusFemale',
    # 'GunPointOldVersusYoung',
    # 'PowerCons',
    # 'Coffee',
    # 'Ham',
    # 'Herring',
    # 'Lightning2',
    # 'MoteStrain',
    # 'PhalangesOutlinesCorrect',
    # 'ProximalPhalanxOutlineCorrect',
    # 'ShapeletSim',
    # 'SonyAIBORobotSurface1',
    # 'SonyAIBORobotSurface2',
    # 'ToeSegmentation1',
    # 'ToeSegmentation2',
    # 'HouseTwenty'
]

In [3]:
dataset = 'Yoga'

x_train, y_train = load_UCR_UEA_dataset(name=dataset, split='train')
# Since the features from the sktime are instatiated as objects we have to manually convert them
y_train = np.array(y_train, dtype=np.int32)

sequence_length = x_train.values[0][0].shape[0]

x_test, y_test = load_UCR_UEA_dataset(name=dataset, split='test')
y_test = np.array(y_test, dtype='int32') # Fixing the labels type

x_train_transformed = []
for val in x_train.values:
    x_train_transformed.append(val[0].tolist())
x_train = np.array(x_train_transformed)

x_test_transformed = []
for val in x_test.values:
    x_test_transformed.append(val[0].tolist())
x_test = np.array(x_test_transformed)

x_train = np.expand_dims(x_train_transformed, axis=1)
x_test = np.expand_dims(x_test_transformed, axis=1)

train_dataset = BaseDataset(x=x_train, y=y_train)
train_loader = DataLoader(train_dataset, batch_size=32)

In [4]:
model = ConvOCC(
    in_channels=1,
    in_features=sequence_length,
    latent_dim=32,
    radius=0.35,
    learning_rate=1e-6
)

model.center = torch.tensor([0. for _ in range(32)])

model.load_pretrained_weights(
    f'../../../pretrain/representation/{dataset}/conv_autoencoder-l32.pt'
)

In [5]:
class LitProgressBar(pl.callbacks.TQDMProgressBar):

    def init_validation_tqdm(self):
        bar = tqdm(
            disable=True,
        )
        return bar

In [6]:
x_train_ = x_train[y_train == 1]
y_train_ = y_train[y_train == 1]

train_dataset = BaseDataset(x=x_train, y=y_train)
train_loader = DataLoader(train_dataset, batch_size=32)

# model._init_center(train_loader)

occ_labels = [1 if x == 1 else -1 for x in y_test]
        
test_dataset = BaseDataset(x=x_test, y=np.array(occ_labels))
test_loader = DataLoader(test_dataset, batch_size=32)

trainer = pl.Trainer(
    max_epochs=300,
    accelerator='gpu',
    devices=-1,
    callbacks=[LitProgressBar()]
)

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | e    | Sequential | 14.3 M
------------------------------------
14.3 M    Trainable params
0         Non-trainable params
14.3 M    Total params
57.041    Total estimated model params size (MB)


Epoch 108:  45%|███▏   | 47/104 [00:00<00:00, 64.92it/s, loss=0.000311, v_num=26, train_loss=0.000, val_f1_score=0.623]

In [7]:
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 94it [00:01, 83.96it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_f1_score       │    0.6188875239148549     │
└───────────────────────────┴───────────────────────────┘

[{'test_f1_score': 0.6188875239148549}]